In [7]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.inspection import inspect
from config import username, password, host, port, protocol

### Extract CSVs into DataFrames

In [8]:
premise_file = "../SAMPLE_PROJECT 2/LicensePremise.csv"
premise_df = pd.read_csv(premise_file)
premise_df.head()

,License Serial Number,Premises Name,License Certificate Number,License Received Date,County ID Code
0,1311660,CANA ARRIBA GROCERY NO 2 INC,NaN,2018-06-29T00:00:00,0
1,1310023,JOHANA GROCERY & DELI CORP,NaN,2018-04-27T00:00:00,0
2,1310024,TKO BEVERAGES LLC,NaN,2018-04-27T00:00:00,1
3,1310024,TKO BEVERAGES LLC,NaN,2018-04-27T00:00:00,1
4,1311663,181 LEXINGTON AVENUE BBQ LLC,NaN,2018-06-29T00:00:00,2


In [9]:
county_file = "../SAMPLE_PROJECT 2/CountyLicenseCount.csv"
county_df = pd.read_csv(county_file)
county_df.head()

,ID,County Name (Licensee),County ID Code,License Count
0,0,ALBANY,5,77
1,1,ALLEGANY,59,4
2,2,BRONX,0,104
3,3,BROOME,35,14
4,4,CATTARAUGUS,41,9


### Transform premise DataFrame

In [10]:
# Create a filtered dataframe from specific columns
premise_cols = ["License Serial Number", "Premises Name", "County ID Code"]
premise_transformed= premise_df[premise_cols].copy()

# Rename the column headers
premise_transformed = premise_transformed.rename(columns={"License Serial Number": "id",
                                                          "Premises Name": "premise_name",
                                                          "County ID Code": "county_id"})

# Clean the data by dropping duplicates and setting the index
premise_transformed.drop_duplicates("id", inplace=True)
premise_transformed.set_index("id", inplace=True)

premise_transformed.head()

,premise_name,county_id
id,,
1311660,CANA ARRIBA GROCERY NO 2 INC,0
1310023,JOHANA GROCERY & DELI CORP,0
1310024,TKO BEVERAGES LLC,1
1311663,181 LEXINGTON AVENUE BBQ LLC,2
1310029,AZIZ DELI & GRILL CORP,1


### Transform county DataFrame

In [11]:
county_cols = ["ID", "County Name (Licensee)", "County ID Code", "License Count"]
county_transformed = county_df[county_cols].copy()

# Rename the column headers
county_transformed = county_transformed.rename(columns={"ID": "id",
                                                         "County Name (Licensee)": "county_name",
                                                         "License Count": "license_count",
                                                         "County ID Code": "county_id"})

# Set index
county_transformed.set_index("id", inplace=True)

county_transformed.head()

,county_name,county_id,license_count
id,,,
0,ALBANY,5,77
1,ALLEGANY,59,4
2,BRONX,0,104
3,BROOME,35,14
4,CATTARAUGUS,41,9


### Create database connection

In [12]:
database_name="customer_db"
rds_connection_string = f'{protocol}://{username}:{password}@{host}:{port}/{database_name}'
engine = create_engine(rds_connection_string)
inspector = inspect(engine)

OperationalError: (psycopg2.OperationalError) could not translate host name "ce61395!" to address: nodename nor servname provided, or not known

(Background on this error at: https://sqlalche.me/e/14/e3q8)

In [ ]:
# Confirm tables
inspector.get_table_names()

['customer_name', 'customer_location']

### Load DataFrames into database

In [ ]:
premise_transformed.to_sql(name='premise', con=engine, if_exists='append', index=True)

In [ ]:
county_transformed.to_sql(name='county', con=engine, if_exists='append', index=True)